# Iteration Workflow

If you are updating a network diagram based on changes made from an earlier session, use this workflow.

If you are looking at a network for the first time, you should try the [Exploratory Workflow](./exploratory_workflow.ipynb).

## To create an updated structure after making new commits:

* Run `lein-topology` to generate the updated edge list.
* Copy the previous position file to use as a starting point for the next visualization.
* Open Cytoscape or destroy existing collections if Cytoscape is already running.
* Run all cells to load the visualization.
* [Save the new layout if you make changes](http://localhost:8888/notebooks/lein-topology.ipynb#Save-the-updated-layout-coordinates-if-you-make-changes).

In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil

import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

import igraph as igraph
import pandas as pd
import json
import numpy as np

import sand.io as io
import sand.graph as sg
import sand.cytoscape.positions as scp
import sand.cytoscape.themes as themes

<IPython.core.display.Javascript object>

## Verify that Cytoscape is running and get the current version of Cytoscape.

_Sending http requests is always an option if a feature is missing from the py2cytoscape wrapper._

In [2]:
import requests

# Basic Setup
PORT_NUMBER = 1234
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'

# Header for posting data to the server as JSON
HEADERS = {'Content-Type': 'application/json'}

version_url = BASE + 'version'
response = requests.get(version_url)

In [3]:
print(json.dumps(response.json(), indent=2))

{
  "cytoscapeVersion": "3.4.0", 
  "apiVersion": "v1"
}


## Extract the subgraph of local namespaces 

There are some analyses where it will be useful to see all the vertices. For the high-level architecture diagram, we can focus on the functions local to the library's namespaces. We'll also keep functions used for output.

In [4]:
commit = "57af741"
network_collection_name = "lein-topology"

In [5]:
# NB: If you're playing along at home, change this to the directory containing your data:
%cd "~/dev/lein-topology"

/Users/bnorton/dev/lein-topology


In [6]:
data_path = "./data/" + network_collection_name + "-" + commit
edge_file = data_path + ".csv"
positions_file = data_path + "-positions.csv"

In [7]:
edgelist = io.csv_to_edgelist(edge_file)
g = sg.edgelist_to_igraph(edgelist)

In [8]:
g.summary()

'IGRAPH D-W- 107 206 -- \n+ attr: label (v), weight (e)'

In [9]:
# List all patterns of vertex names that we want to keep:
names_to_keep = ['topology', 'clojure.core/*err*', 'clojure.core/println']

In [10]:
lv = g.vs(lambda v: any(match in v['label'] for match in names_to_keep))
lg = g.subgraph(lv)

In [11]:
lg.summary()

'IGRAPH D-W- 26 26 -- \n+ attr: label (v), weight (e)'

## Load into Cytoscape with a default layout

In [12]:
# Create py2cytoscape client
cy = CyRestClient()

In [13]:
# Start by loading the full network
network = cy.network.create_from_igraph(lg, name=commit, collection=network_collection_name)

In [14]:
network_id = network.get_id()
network_id

300

In [15]:
# Apply layout
cy.layout.apply(name='force-directed', network=network)

## Customize the style

Use one of the included themes, or build your own.

In [16]:
style = cy.style.create('Ops')
style.update_defaults(themes.ops.settings)

# Map the label property in the igraph data to Cytoscape's NODE_LABEL visual property
style.create_passthrough_mapping(column='label', vp='NODE_LABEL', col_type='String')

cy.style.apply(style, network)

## Load layout coordinates from a previous session

In [17]:
scp.layout_from_positions_csv(network, positions_file, cy)

## Save the updated layout coordinates if you make changes

One benefit of this workflow is the ability to make changes manually to the network layout in Cytoscape.
After making changes, save the coordinates for a later session:

In [19]:
scp.positions_to_csv(network=network, path=positions_file)

You can now safely close Cytoscape.

### Generate an SVG export

To generate an SVG based on the current view, visit the following:

In [18]:
print("http://localhost:1234/v1/networks/" + str(network_id) + "/views/first.svg")

http://localhost:1234/v1/networks/300/views/first.svg
